In [ ]:
import numpy as np

def fft_recursive(x):
    """
    x: Input array (1D) berupa bilangan kompleks atau real
    """
    N = len(x)
    
    # 1. Base Case: Jika N=1, return x
    if N <= 1:
        return x
    
    #Pisah Genap dan Ganjil
    even = fft_recursive(x[0::2])
    odd  = fft_recursive(x[1::2])
    
    k = np.arange(N // 2)
    T = np.exp(-2j* np.pi * k / N) * odd
    
    return np.concatenate([even + T, even - T])

# --- CONTOH PENGGUNAAN ---

# Buat sinyal dummy (misal: kombinasi 2 gelombang sinus)
t = np.linspace(0, 1, 1024)
freq1, freq2 = 5, 20 # 5 Hz dan 20 Hz
signal = np.sin(2 * np.pi * freq1 * t) + 0.5 * np.sin(2 * np.pi * freq2 * t)

# Jalankan FFT buatan sendiri
fft_result = fft_recursive(signal)

# Bandingkan dengan NumPy asli (untuk verifikasi)
fft_numpy = np.fft.fft(signal)

print(f"Apakah hasilnya sama? {np.allclose(fft_result, fft_numpy)}")

Apakah hasilnya sama? True


In [24]:
a=fft_recursive([1,2,4,7])

In [43]:
a

array([14.+0.j, -3.+5.j, -4.+0.j, -3.-5.j])

In [48]:
def inverse_fft_recursive(x):
    N = len(x)
    if N<=1:
        return x
    
    even = inverse_fft_recursive(x[0::2])
    odd = inverse_fft_recursive(x[1::2])

    k = np.arange(N//2)
    T = np.exp(2j*np.pi * k / N) *odd

    return np.concatenate([even+T,even-T])

In [49]:
def inverse_fft(x):
    ans = inverse_fft_recursive(x)
    ans = ans/(len(x))
    return ans

In [50]:
inverse_fft(a)

array([1.+0.0000000e+00j, 2.-6.8963755e-17j, 4.+0.0000000e+00j,
       7.+6.8963755e-17j])

In [54]:
[1,2,3,4] * [2,3,4,5]

TypeError: can't multiply sequence by non-int of type 'list'

In [141]:
def multiplypolinoms(p1,p2):
    # p1 and p2 contains array of coefficients
    target_length = len(p1) + len(p2) -1

    n = 1
    while(n < target_length) :
        n*=2
    new_p1 = np.zeros(n)
    new_p2 = np.zeros(n)

    new_p1[:len(p1)] = p1
    new_p2[:len(p2)] = p2

    # 1. Convert to list of value DFT
    fft_p1 = fft_recursive(new_p1)
    fft_p2 = fft_recursive(new_p2)
    multiple = fft_p1 * fft_p2
    # for i in range (len(p1)):
    #     multiple[i] = fft_p1[i] * fft_p2[i]
    coeff = inverse_fft(multiple)
    coeff = coeff[:n-1]
    return coeff

In [142]:
import numpy as np

In [143]:
a = np.random.randint(300,size = int(2048))
b = np.random.randint(300,size = int(2048))

In [144]:
a,b

(array([257,  90,  97, ..., 286, 116, 109], shape=(2048,), dtype=int32),
 array([295,  82, 250, ..., 290, 179,  36], shape=(2048,), dtype=int32))

In [145]:
fft_style = multiplypolinoms(a,b)

In [146]:
fft_style

array([ 75815.        -1.78706614e-08j,  47624.        -1.67251070e-08j,
       100245.        -1.77896387e-08j, ...,
        62670.        +1.54104017e-09j,  23686.99999999-3.81840458e-09j,
         3924.        -3.86435755e-09j], shape=(4095,))

In [147]:
def manual_multiply_polinoms(p1,p2):
    n = len(p1) + len(p2) -1
    res = np.zeros(n)
    for i in range(len(p1)) :
        for j in range(len(p2)) :
            res[i+j] += p1[i] * p2[j]
    
    return res

In [148]:
from sklearn.metrics import mean_absolute_error


In [149]:
manual = manual_multiply_polinoms(a,b)

In [150]:
manual

array([ 75815.,  47624., 100245., ...,  62670.,  23687.,   3924.],
      shape=(4095,))

In [151]:
mean_absolute_error(manual,fft_style)

ValueError: Complex data not supported
[ 75815.        -1.78706614e-08j  47624.        -1.67251070e-08j
 100245.        -1.77896387e-08j ...  62670.        +1.54104017e-09j
  23686.99999999-3.81840458e-09j   3924.        -3.86435755e-09j]


In [2]:
def fft2d(image):
    """
    Melakukan FFT 2 Dimensi untuk gambar.
    Input: image (H x W)
    """
    H, W = image.shape
    
    # Langkah 1: FFT pada setiap Baris (Row-wise)
    rows_fft = np.zeros((H, W), dtype=complex)
    for i in range(H):
        rows_fft[i, :] = fft_recursive(image[i, :])
        
    # Langkah 2: FFT pada setiap Kolom (Column-wise) dari hasil langkah 1
    cols_fft = np.zeros((H, W), dtype=complex)
    for j in range(W):
        cols_fft[:, j] = fft_recursive(rows_fft[:, j])
        
    return cols_fft